Load packages

In [1]:

import pyomo.environ as pyo
from datetime import date
import pandas as pd
import ast
import numpy as np

Define DA model with Imbalance Reserves

In [2]:
DAIRmodel = pyo.AbstractModel()

#Sets
DAIRmodel.G = pyo.RangeSet(1, 5)
DAIRmodel.S = pyo.RangeSet(1, 5)
DAIRmodel.R = pyo.RangeSet(1, 4)

#Parameters
DAIRmodel.VC = pyo.Param(DAIRmodel.G) #Variable Costs of generators
DAIRmodel.CAP = pyo.Param(DAIRmodel.G) #Capacity of generators
DAIRmodel.RR= pyo.Param(DAIRmodel.G)  #Ramp rate
DAIRmodel.DEMAND=pyo.Param(within=pyo.NonNegativeIntegers) #Demand
DAIRmodel.D1=pyo.Param(within=pyo.NonNegativeIntegers) #linear cost coefficient for demand slack variable
DAIRmodel.D2=pyo.Param(within=pyo.NonNegativeIntegers) #quadratic cost coefficient for demand slack variable
DAIRmodel.REDA=pyo.Param(within=pyo.NonNegativeIntegers) #Renewable day-ahead
DAIRmodel.virtualcost=pyo.Param()
DAIRmodel.VUB=pyo.Param()
DAIRmodel.VLB=pyo.Param()

#Parameters related to IR
DAIRmodel.avup= pyo.Param(DAIRmodel.G) #Capacity bid for IR up
DAIRmodel.avdn= pyo.Param(DAIRmodel.G) #Capacity bid for IR down
DAIRmodel.PEN=pyo.Param(DAIRmodel.R,within=pyo.NonNegativeIntegers) #Scarcity price for IR up
DAIRmodel.PEND=pyo.Param(DAIRmodel.R,within=pyo.NonNegativeReals) #Scarcity price for IR down
DAIRmodel.probTU=pyo.Param(DAIRmodel.R) #probability of scarcity if IR up at level R is unmet
DAIRmodel.probTD=pyo.Param(DAIRmodel.R) #probability of surplus if IR down at level R is unmet

#Parameters related to IR requirement estimation - typically exogenous to model
DAIRmodel.prob=pyo.Param(DAIRmodel.S)
DAIRmodel.RE = pyo.Param(DAIRmodel.S)

# Decision variables
DAIRmodel.xDA=pyo.Var(DAIRmodel.G, domain=pyo.NonNegativeReals)
DAIRmodel.rgDA=pyo.Var(domain=pyo.NonNegativeReals)
DAIRmodel.d=pyo.Var(domain=pyo.NonNegativeReals)
DAIRmodel.arb=pyo.Var()
#Decision variables for IR
DAIRmodel.isu  = pyo.Var(DAIRmodel.G, domain=pyo.NonNegativeReals) #IR up provided by supplier g
DAIRmodel.isd  = pyo.Var(DAIRmodel.G, domain=pyo.NonNegativeReals) #IR down provided by supplier g
DAIRmodel.siu  = pyo.Var(DAIRmodel.R, domain=pyo.NonNegativeReals) #Shortfall of IR up
DAIRmodel.sid  = pyo.Var(DAIRmodel.R, domain=pyo.NonNegativeReals) #Shortfall of IR down

#Decision variables for IR supply by renewables
DAIRmodel.iupre =pyo.Var(domain=pyo.NonNegativeReals)
DAIRmodel.idnre =pyo.Var(domain=pyo.NonNegativeReals)

def obj_expression(m):
    return sum(m.VC[g]*m.xDA[g] for g in m.G)\
+sum(m.probTU[r]*m.PEN[r]*m.siu[r] for r in m.R)\
+sum(m.probTD[r]*m.PEND[r]*m.sid[r] for r in m.R)\
+sum(m.avup[g]*m.isu[g] for g in m.G)\
+sum(m.avdn[g]*m.isd[g] for g in m.G)+m.D1 *m.d+ m.D2 *m.d*m.d +m.virtualcost*m.arb
DAIRmodel.OBJ = pyo.Objective(rule=obj_expression)

def DA_energy_balance(model):
    return sum(model.xDA[g] for g in model.G)+ model.rgDA +model.arb +model.d == model.DEMAND
DAIRmodel.Con3 = pyo.Constraint(expr=DA_energy_balance) 

def DA_IRup_balance(model):
    return sum(model.isu[g] for g in model.G) +sum(model.siu[t] for t in model.R) +model.iupre >= np.mean(np.array([model.RE[s] for s in model.S]))-min(model.RE[s] for s in model.S)
DAIRmodel.Con4UP= pyo.Constraint(rule=DA_IRup_balance)

def shortage_IR_up(model,r):
    return model.siu[r]<=max(0, min(np.mean(np.array([model.RE[s] for s in model.S])),model.RE[r+1])-model.RE[r])
DAIRmodel.Con4UPstep= pyo.Constraint(DAIRmodel.R,rule=shortage_IR_up)

def DA_IRdn_balance(model):
    return sum(model.isd[g] for g in model.G)+sum(model.sid[t] for t in model.R) +model.idnre>= max(model.RE[s] for s in model.S)-np.mean(np.array([model.RE[s] for s in model.S]))
DAIRmodel.Con4DN = pyo.Constraint(rule=DA_IRdn_balance)

def shortage_IR_dn(model,r):
    return model.sid[r]<=max(0, model.RE[r+1]-max(np.mean(np.array([model.RE[s] for s in model.S])),model.RE[r]))
DAIRmodel.Con4DNstep= pyo.Constraint(DAIRmodel.R,rule=shortage_IR_dn)



def RE_dn(model):
    return model.idnre<= model.rgDA
DAIRmodel.Con5 = pyo.Constraint(expr=RE_dn)

def RE_up(model):
    return model.iupre +model.rgDA<= np.mean(np.array([model.RE[s] for s in model.S]))
DAIRmodel.Con6 = pyo.Constraint(expr=RE_up)                                                                                                            
                                                                                                            
                                                                                               
def RRUP(model,g):
    return model.isu[g]<=model.RR[g]
DAIRmodel.Con10UP = pyo.Constraint(DAIRmodel.G, rule=RRUP)  

def RRDN(model,g):
    return model.isd[g] <=model.RR[g]
DAIRmodel.Con10DN = pyo.Constraint(DAIRmodel.G, rule=RRDN) 

def DA_capacity_cons(model, g):
    return model.xDA[g] +model.isu[g]<= model.CAP[g]
DAIRmodel.Con11 = pyo.Constraint(DAIRmodel.G, rule=DA_capacity_cons)

def DA_down_cons(model, g):
    return  model.isd[g] <= model.xDA[g]
DAIRmodel.Con12 = pyo.Constraint(DAIRmodel.G, rule=DA_down_cons)

def VB_UB(model):
    return model.arb<=model.VUB
DAIRmodel.Con13 = pyo.Constraint(rule=VB_UB)

def VB_LB(model):
    return model.arb>=model.VLB
DAIRmodel.Con14 = pyo.Constraint(rule=VB_LB)


DAIRmodel.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

Stochastic Program that is used to simulate the impact of DA decisions on RT

In [3]:
RTsim = pyo.AbstractModel()
#Sets
RTsim.S = pyo.RangeSet(1, 5)
RTsim.G = pyo.RangeSet(1, 5)

#Parameters
RTsim.VC = pyo.Param(RTsim.G)
RTsim.CAP = pyo.Param(RTsim.G)
RTsim.RR= pyo.Param(RTsim.G)

RTsim.prob=pyo.Param(RTsim.S)
RTsim.RE = pyo.Param(RTsim.S)
RTsim.DEMAND=pyo.Param(within=pyo.NonNegativeIntegers)
RTsim.D1=pyo.Param(within=pyo.NonNegativeIntegers)
RTsim.D2=pyo.Param(within=pyo.NonNegativeIntegers)
RTsim.xDA=pyo.Param(RTsim.G, domain=pyo.NonNegativeReals)
RTsim.REDA=pyo.Param( domain=pyo.NonNegativeReals)
RTsim.PEN=pyo.Param(within=pyo.NonNegativeIntegers)
RTsim.PENDN=pyo.Param()
RTsim.ARB=pyo.Param()
RTsim.DAdr=pyo.Param()

# Variables
RTsim.xup = pyo.Var(RTsim.S, RTsim.G, domain=pyo.NonNegativeReals)
RTsim.xdn = pyo.Var(RTsim.S, RTsim.G, domain=pyo.NonNegativeReals)
RTsim.d= pyo.Var(RTsim.S, domain=pyo.NonNegativeReals)
RTsim.rgup = pyo.Var(RTsim.S, domain=pyo.NonNegativeReals)
RTsim.rgdn = pyo.Var(RTsim.S, domain=pyo.NonNegativeReals)
RTsim.sdup = pyo.Var(RTsim.S, domain=pyo.NonNegativeReals)
RTsim.sddn = pyo.Var(RTsim.S, domain=pyo.NonNegativeReals)

#Objective
def obj_expression(m):
    return sum(m.prob[s]*m.VC[g]*(m.xup[s,g]-m.xdn[s,g]) for g in m.G for s in m.S)\
-m.PENDN*sum(m.prob[s]*m.sdup[s] for s in m.S)+m.PEN*sum(m.prob[s]*m.sddn[s] for s in m.S)\
+sum(m.prob[s]*(m.D1*(m.DAdr+m.d[s])+m.D2*(m.DAdr+m.d[s])*(m.DAdr+m.d[s])) for s in m.S)\
-sum(m.prob[s]*(m.D1*(m.DAdr)+m.D2*(m.DAdr)*(m.DAdr)) for s in m.S)
RTsim.OBJ = pyo.Objective(rule=obj_expression)

#Constraints
def RT_energy_balance(model,s):
    return sum(model.xup[s,g]-model.xdn[s,g] for g in model.G)+ model.rgup[s]-model.rgdn[s]+model.d[s] == model.ARB

def RT_capacity_cons(model,s, g):
    return model.xDA[g]+model.xup[s,g] <= model.CAP[g]

def RT_capacity_min(model,s, g):
    return model.xDA[g]-model.xdn[s,g] >=0

def RT_RE_availability(model,s):
    return model.rgup[s]-model.rgdn[s]+model.sdup[s]-model.sddn[s] ==model.RE[s]-model.REDA 

def RT_ramp_up(model,s,g):
    return model.xup[s,g] <=model.RR[g]

def RT_ramp_dn(model,s,g):
    return model.xdn[s,g] <= model.RR[g]

# Numbering of constraints follows the RT problem
RTsim.Con3 = pyo.Constraint(RTsim.S, rule=RT_energy_balance)
RTsim.Con4 = pyo.Constraint(RTsim.S, rule= RT_RE_availability)
RTsim.Con5up = pyo.Constraint(RTsim.S,RTsim.G, rule=RT_ramp_up)
RTsim.Con5dn = pyo.Constraint(RTsim.S, RTsim.G,rule=RT_ramp_dn)
RTsim.Con6 = pyo.Constraint(RTsim.S,RTsim.G, rule=RT_capacity_cons)
RTsim.Con7 = pyo.Constraint(RTsim.S,RTsim.G, rule=RT_capacity_min)

#Record duals
RTsim.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

Use CPLEX to solve the problem [quadratic objective function due to elastic demand]

In [4]:
opt = pyo.SolverFactory('cplex',executable='C:/Program Files/IBM/ILOG/CPLEX_Studio2211/cplex/bin/x64_win64/cplex')

In [5]:
input_data=pd.read_csv('Input_files/IR_input_sectionV.csv',index_col=0)
Summary_cost=pd.DataFrame()
Summary_price_convergence=pd.DataFrame()
Summary_margin_intermittency=pd.DataFrame()
Summary_costD=pd.DataFrame()
Summary_costCURT=pd.DataFrame()
for l in input_data.columns:
        dataDA=input_data[l].to_dict()
        for k in range(0,len(input_data[l])):
            dataDA[input_data.index[k]]=ast.literal_eval(input_data[l][input_data.index[k]])
        dataDA = {None: dataDA}
        i = DAIRmodel.create_instance(dataDA)
        opt.solve(i)
        #Values to pass from DA to RT module
        Energy=pd.DataFrame()
        E_grid_data = {(G): v.value for (G), v in i.xDA.items()}
        Energy['en']= pd.DataFrame.from_dict(E_grid_data, orient="index", columns=["en"])
        Energy['en']=round(Energy['en'],2)
        temp=i.rgDA.value
        Total=pd.DataFrame()
        Total.at['cost','DA']=sum(np.multiply(np.array([i.VC[k] for k in i.G]),np.array([i.xDA[g].value for g in i.G])))
        Total.at['price','DA']=i.dual[i.Con3]
        E_grid_data = {(G): v.value for (G), v in i.isu.items()}
        df = pd.DataFrame.from_dict(E_grid_data, orient="index", columns=["isu"])
        E_grid_data = {(G): v.value for (G), v in i.isd.items()}
        df ['isd']= pd.DataFrame.from_dict(E_grid_data, orient="index", columns=["isd"])
        Prices=pd.DataFrame()
        Prices.at[0,'up']=i.dual[i.Con4UP]
        Prices.at[0,'down']=i.dual[i.Con4DN]
        Gross_margins=pd.DataFrame()
        Gross_margins["en"]=np.multiply(i.dual[i.Con3]-np.array([i.VC[k] for k in i.G]), Energy['en'])
        Gross_margins.at[0,"en"]=i.rgDA.value*i.dual[i.Con3]
        Gross_margins["up"]=i.dual[i.Con4UP]*df['isu']
        Gross_margins["down"]=i.dual[i.Con4DN]*df['isd']
            
        dataRT = {None:{
            'RE': dataDA[None]["RE"],
            'CAP': dataDA[None]["CAP"],
            'VC': dataDA[None]["VC"],
            'DEMAND': dataDA[None]["DEMAND"],
            'D1': dataDA[None]["D1"],
            'D2': dataDA[None]["D2"],
            'prob':{1: 0.2, 2:0.2, 3:0.2, 4:0.2, 5:0.2},
              'RR': dataDA[None]["RR"],
            'xDA':Energy['en'],
             'PEN':{None: dataDA[None]["PEN"][1]},
             'PENDN':{None: dataDA[None]["PEND"][1]},
             'REDA':{None: temp},
             'ARB':{None: i.arb.value},
             'DAdr':{None: i.d.value}
            }}
        iRT = RTsim.create_instance(dataRT)
        opt.solve(iRT)
        RTmargins=pd.DataFrame()
        for s in iRT.S:  
            RTmargins[s]=(np.multiply(iRT.dual[iRT.Con3[s]]-iRT.prob[s]*np.array([iRT.VC[k] for k in iRT.G]),np.array([iRT.xup[s,k].value-iRT.xdn[s,k].value  for k in i.G])))
            Total.at['cost',s]=iRT.prob[s]*sum(np.multiply(np.array([i.VC[k] for k in i.G]),np.array([iRT.xup[s,g].value-iRT.xdn[s,g].value for g in iRT.G])))
            Total.at['price',s]=iRT.dual[iRT.Con3[s]]
            Total.at['unmet_demand',s]=iRT.prob[s]*(iRT.D1*(i.d.value+iRT.d[s].value)+iRT.D2*(iRT.d[s].value+i.d.value)*(i.d.value+iRT.d[s].value))
            Total.at['curtail cost',s]=iRT.prob[s]*(iRT.PENDN*iRT.sdup[s].value)
        price_convergence=Total.at['price','DA']-Total[Total.columns[1:]].loc['price'].sum()
        Total_margin=pd.DataFrame()
        for k in range(1,6):
             Total_margin[k]=iRT.prob[k]*Gross_margins.sum(axis=1)[0:5].values+RTmargins[k]
             Total_margin.at['RE',k]=(iRT.dual[iRT.Con3[k]]*(iRT.rgup[k].value-iRT.rgdn[k].value)) +iRT.prob[k]*temp*i.dual[i.Con3]
        writer = pd.ExcelWriter('Test_output_files/IR_'+l+"_v"+date.today().strftime("%Y_%m_%d")+'revision.xlsx')      
        round(Total,2).to_excel(writer,'System_metrics')
        round(RTmargins,2).to_excel(writer,'RTmargins')
        round(Gross_margins,2).to_excel(writer,'Grossmargins')
        round(df,2).to_excel(writer,'IR_supply_AWARDS')
        round(Energy,2).to_excel(writer,'DA_energy')
        round(Total_margin,2).to_excel(writer,'Totalmargins')
        round(Prices,2).to_excel(writer,'Prices')
        pd.DataFrame(dataDA).to_excel(writer,'input_DA')
        pd.DataFrame(dataRT).to_excel(writer,'input_RT')
        writer.close()
        Summary_cost[l]=Total.loc['cost'].transpose()
        Summary_costD[l]=Total.loc['unmet_demand'].transpose()
        Summary_price_convergence[l]=Total.loc['price'].transpose()
        Summary_margin_intermittency[l]=Total_margin.apply(lambda row: sum(row>0.01),axis=1)
        Summary_costCURT[l]=Total.loc['curtail cost'].transpose()

In [6]:
Prices

,up,down
0,30.000001,0.0


In [ ]:
i.dual[i.Con4UP]